# Libs

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

from graph_representation import *

# Create Grid Network

In [ ]:
G = nx.DiGraph()

In [ ]:
grid_size = (4,4)

node_list = []
for i in range(grid_size[0]):
    for j in range(grid_size[1]):
        node_list.append((i,j))

In [ ]:
for node in node_list:
    G.add_node(node)

In [ ]:
max_idx = max(grid_size) - 1
for node in node_list:
    row, col = node
    if row > 0:
        G.add_edge(node,(row - 1,col))
        G.add_edge((row - 1,col),node)
    if col > 0:
        G.add_edge(node,(row,col - 1))
        G.add_edge((row,col - 1),node)
    if row < max_idx:
        G.add_edge(node,(row + 1,col))
        G.add_edge((row + 1,col),node)
    if col < max_idx:
        G.add_edge(node,(row,col + 1))
        G.add_edge((row,col + 1),node)

In [ ]:
nx.draw(G,node_size=100,node_color='b',edge_color='k')

# Transform into Movement Network

In [ ]:
def cal_direction(edge_idx):
    '''
    edge_idx: [row_diff, col_diff]
    # [1,0] --> heading South
    # [-1,0] --> heading North
    # [0,1] --> heading East
    # [0,-1] --> heading West
    '''
    if edge_idx == [1,0]:
        return 'South'
    elif edge_idx == [-1,0]:
        return 'North'
    elif edge_idx == [0,1]:
        return 'East'
    elif edge_idx == [0,-1]:
        return 'West'
    else:
        print('Wrong edge index!')

In [ ]:
margin_edges = []
for row in range(max_idx + 1):
    margin_edges.append(((row, -1),(row, 0)))
    margin_edges.append(((row,max_idx + 1),(row,max_idx)))
for col in range(max_idx + 1):
    margin_edges.append(((-1,col),(0,col)))
    margin_edges.append(((max_idx + 1,col),(max_idx,col)))

In [ ]:
G_mv = nx.DiGraph()
direction_to_mv_origin = {'nbd': {'South': [-1, 0, 1], 'East': [0, 1], 'West': [-1, 0]},
                          'sbd': {'North': [-1, 0, 1], 'East': [0, -1], 'West': [0, 1]},
                          'wbd': {'South': [-1, 0], 'North': [0, 1], 'East': [-1, 0, 1]},
                          'ebd': {'South': [1, 0], 'North': [0, -1], 'West': [-1, 0, 1]},
                          'nwbd': {'South': [0, -1], 'East': [0, 1]},
                          'nebd': {'South': [0, 1], 'West': [-1, 0]},
                          'swbd': {'North': [0, 1], 'East': [0, -1]},
                          'sebd': {'North': [0, -1], 'West': [0, 1]}}
direction_to_mv_destination = {'nwbd': {'North': [1], 'West': [-1]},
                               'nebd': {'North': [-1], 'East': [1]},
                               'swbd': {'South': [-1], 'West': [1]},
                               'sebd': {'South': [1], 'East': [-1]}}

for new_node in list(G.edges) + margin_edges:
    for mv in [-1,0,1]:
        G_mv.add_node(tuple(list(new_node) + [mv]))

In [ ]:
fn_row_minus = lambda a:(a[0] - 1,a[1])
fn_row_plus = lambda a:(a[0] + 1,a[1])
fn_col_minus = lambda a:(a[0],a[1] - 1)
fn_col_plus = lambda a:(a[0],a[1] + 1)

In [ ]:
for node in G_mv.nodes:
    
    direction = cal_direction([node[1][i] - node[0][i] for i in range(2)])
    
    # South, -1 --> col + 1; South, 0 --> row + 1; South, 1 --> col - 1
    # North, -1 --> col - 1; North, 0 --> row - 1; North, 1 --> col + 1
    # East, -1 --> row - 1; East, 0 --> col + 1; East, 1 --> row + 1
    # West, -1 --> row + 1; West, 0 --> col - 1; West, 1 --> row - 1

    from_node = node[1]
    if (direction,node[-1]) in [('South',0),('East',1),('West',-1)]:
        to_node = fn_row_plus(node[1])
    elif (direction,node[-1]) in [('North',0),('East',-1),('West',1)]:
        to_node = fn_row_minus(node[1])
    elif (direction,node[-1]) in [('South',-1),('North',1),('East',0)]:
        to_node = fn_col_plus(node[1])
    elif (direction,node[-1]) in [('South',1),('North',-1),('West',0)]:
        to_node = fn_col_minus(node[1])

    if to_node not in G.nodes:
        continue
    else:
        connecting_node = []
        for mv in [-1,0,1]:
            trial_node = tuple([from_node, to_node] + [mv])
            if trial_node in G_mv.nodes:
                connecting_node.append(trial_node)
        for i in range(len(connecting_node)):
            G_mv.add_edge(node,connecting_node[i],weight=0.1*np.random.rand())

In [ ]:
degree_dict = dict(G_mv.degree)
rmv_node = []
for key in degree_dict.keys():
    if degree_dict[key] == 0:
        rmv_node.append(key)
        G_mv.remove_node(key)

In [ ]:
plt.figure(figsize=(8,8))
nx.draw(G_mv,node_size=30,node_color='r',edge_color='k',pos=nx.kamada_kawai_layout(G_mv))

# Lib Test

In [ ]:
from graph_representation import *

In [ ]:
node_dim = np.zeros((9,2))
edge_dim = np.zeros((9,2))
for size in range(2,11):
    G = create_grid_network(size)
    G_mv = movement_network_transform(G)
    node_dim[size - 2,:] = [len(G.nodes),len(G_mv.nodes)]
    edge_dim[size - 2,:] = [len(G.edges),len(G_mv.edges)]

In [ ]:
plt.rc('font',family='Times New Roman',size=12)
plt.plot(node_dim[:,0],node_dim[:,1],marker='o',c='b')
plt.xlabel('Grid Network Node Number',fontdict={'size':14})
plt.ylabel('Movement Network Node Number',fontdict={'size':14})
k = (node_dim[:,1] / node_dim[:,0]).mean()
plt.text(40,800,'k = {:.2f}'.format(k),fontdict={'color':'b','size':15,'style':'italic'})

In [ ]:
plt.rc('font',family='Times New Roman',size=12)
plt.plot(edge_dim[:,0],edge_dim[:,1],marker='^',c='r')
plt.xlabel('Grid Network Edge Number',fontdict={'size':14})
plt.ylabel('Movement Network Edge Number',fontdict={'size':14})
k = (edge_dim[:,1] / edge_dim[:,0]).mean()
plt.text(150,2000,'k = {:.2f}'.format(k),fontdict={'color':'r','size':15,'style':'italic'})

# Load Path Flow

In [ ]:
G = create_grid_network(size=5)
G_mv = movement_network_transform(G)

In [ ]:
# select od

In [ ]:
# select shortest path

# generate path flow

# generate movement flow (node weight)

# generate movement pair flow (edge weight)

# Community Detection

In [ ]:
size = 3
G = create_grid_network(size)
G_mv = movement_network_transform(G)

In [ ]:
from networkx.algorithms.community import asyn_lpa_communities, girvan_newman

In [ ]:
gen = girvan_newman(G_mv)
node_group = {idx:list(i) for idx,i in enumerate(gen)}

In [ ]:
node_class = {}
c = 0
for group in node_group:
    for node in group:
        node_class[node] = c
    c += 1

In [ ]:
nx.draw(G_mv,node_color=list(node_class.values()))